In [2]:
# ---------- IMPORTAÇÃO DAS BIBLIOTECAS ---------- #

import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import seaborn as sns
import time

# ---------- CONEXÃO COM O BANCO DE DADOS ---------- #
# Parâmetros da Conexão
host    = 'lincpo.com.br'
port    = '2035'
dbname  = 'censo'
dbtable = 'censo_2000'
user    = 'prevdb_user'
pwd     = 'pr3v'

# Conexão com o Banco de Dados
conn = psycopg2.connect("host='{}' port={} dbname='{}'user={} password={}"
         .format(host, port, dbname, user, pwd))

if conn == -1:
    print("Não foi possível conectar a Base de Dados")
else:
    print("Conectado com sucesso")

Conectado com sucesso


# **Censo Demográfico 2000 - Análise dos Benefícios e Indíce de Gini**

* Cálculo do indíce de Gini (renda percapita e percapita domiciliar) nas base de dados dos censos de: 
     * **1991** (?)
     * **2000**
     * **2010**
* Base de Dados disponível em: ftp://ftp.ibge.gov.br/Censos/
* Resultados divulgados pelo IBGE disponível em: http://www.atlasbrasil.org.br/2013/pt/consulta/
* Unidade de amostra: **Estados**
* Análise estátisca acerca da situação dos benefícios (aposentadorias e pensões)
* Anuário estatistico da previdência (ano 2000?): http://www.previdencia.gov.br/a-previdencia/politicas-de-previdencia-social/resultados-do-rgps/

## **1. Indíce de Gini**
* Indicador que mede a desigualdade de renda
* G = 1 corresponde a total desigualdade e G = 0 corresponde a total igualdade de renda
* Pode ser cálculado como:

$G = 1 - CurvaLorenz$

ou 

$G = 1 - \sum_{i=1}^{k=n-1} (p_{k+1} - p_{k})(r_{k+1} + r_{k})$
* Onde: 
    - G - Coeficiente de Gini
    - P - Proporção acumulada da "população"
    - R - Proporção acumulada de "renda"
    
* Obs: é necessário organizar **R** de forma crescente: 

$r_{1} ≤ r_{2} ≤ r_{3} ... ≤ r_{i} ≤ ... ≤ r_{n}$

In [3]:
# ---------- CALCULO DO ÍNDICE DE GINI ---------- #

def gini( sql ):
    df = pd.read_sql_query( sql, conn )
    
    # Separa valores em renda e população
    renda = df[ 'renda' ]
    pessoas = df[ 'peso' ]
    
    # Transforma as listas em arrays
    pessoas = np.array( pessoas )
    renda = np.array( renda )

    # Organiza os dados
    aux = np.argsort( [ renda/pessoas for renda, pessoas in zip( renda, pessoas ) ], kind = "stable")
    
    renda = [j for _, j in zip( aux, renda[ aux ] )]
    pessoas = [i for _, i in zip(aux, pessoas[ aux ] )]

    # Soma acumulada de renda e pessoas 
    renda = np.cumsum( renda / sum( renda ) )
    pessoas = np.cumsum( pessoas / sum( pessoas ) )
    
    # Fŕomula do indíce de gini [ Gini = 1 - CurvaLorenz]
    g = 2 * ( 0.5 - np.trapz( renda, pessoas ) )
    
    return g, pessoas, renda

In [4]:
# ---------- PLOT DA CURVA DE LORENZ---------- #

def plotagem( pessoas, renda ):
    plt.plot( pessoas, renda, 'k' )
    plt.plot( [ 0, 1 ], 'r' )
    plt.ylim( 0, 1 ), plt.xlim( 0, 1 )
    plt.fill_between( pessoas, renda, facecolor = 'y' )

    plt.xlabel( 'Proporção de população' )
    plt.ylabel( 'Proporção de renda' )
    plt.title( 'Curva de Lorenz' )
    plt.show()

### **Variáveis de Interesse da Base de Dados**

* Variáveis de domicílios:
    - **v0102** - Código do estado
    - **v0103** - Código município 
    - **v7616** - Rendimento domiciliar
    - **v7100** - Número de moradores do domicílio
    - **p001**  - Peso da amostra associado a quantas vezes o registro se repete
    
* Variáveis de pessoas:
    - **v0102** - Código do estado
    - **v1103** - Código município 
    - **v4512** - (*) Rendimento bruto do trabalho principal
    - **v4525** - (*) Rendimento de todos os trabalhos
    - **v4614** - (*) Rendimento total
    - **v4573** - Rendimento de aposentadorias e pensões
    - **p001**  - Peso da amostra associado a quantas vezes o registro se repete

In [ ]:
# ---------- CALCULO DO ÍNDICE DE GINI PARA RENDA DOMICILIAR - ESTADOS ---------- #

estados = [ "ac", "al", "am", "ap", "ba", "ce", "df", "es", "go", "ma", "mg", "ms", "mt",
          "pa", "pb", "pe", "pi", "pr", "rj", "rn1", "ro", "rr", "rs", "sc", "se", 
          "sp", "to" ]

results = []

for k in estados:
        
    # SQL - índice de gini renda domiciliar  
    sql_RD = "SELECT \
            COALESCE( ( v7616 * p001 ), 0 ) AS renda, ( p001 ) AS peso \
            FROM censo_2000.domicilios_%s" %( k )

    # SQL - índice de gini renda per capita
    sql_RP = "SELECT \
            COALESCE( ( v7616 * p001 ), 0 ) AS renda, ( p001 * v7100 ) AS peso \
            FROM censo_2000.domicilios_%s" %( k )

    # SQL - renda per capita
    sql_Renda = "SELECT \
                ( SUM( v7616 * p001 ) / SUM( v7100 * p001 ) ) AS RP \
                FROM censo_2000.domicilios_%s" %( k )
    
    sql_UF = "SELECT v0102 AS UF FROM censo_2000.domicilios_%s limit 1" %( k )
    
    df_UF = pd.read_sql_query( sql_UF, conn )
    df_Renda = pd.read_sql_query( sql_Renda, conn )
        
    # Ajuste da renda per capita de julho de 2000 para valores em agosto de 2010
    # 99,19% de inflação no INCP
    renda2010 = df_Renda.loc[ 0, 'rp' ] + ( df_Renda.loc[ 0, 'rp' ] * 0.9919037 )
        
    # Obs: A função "gini()" retorna um array com [índice de gini], ...
    # proporção acumulado da população] e [proporção acumulada de renda]
    g_RD = gini( sql_RD )
    g_RP = gini( sql_RP )
        
    # Adiciona valores no array resultado
    # || ID do municipio || Gini RD || Gini RP || Renda2000 || Renda2010 ||
    results.append( [df_UF.loc[ 0, 'uf' ],
                     k,
                     g_RD[ 0 ], 
                     g_RP[ 0 ], 
                     df_Renda.loc[ 0, 'rp' ], 
                     renda2010 ] )

        
# Salva em um arquivo .csv
gini_domicilio = pd.DataFrame( results, columns = 'Cod UF Gini_RD Gini_RP RendaMedia RendaMedia2010'.split() )
gini_domicilio.to_csv( 'results/gini2000_uf.csv', sep = '\t' )
        
print( "Processamento do cálculo do índice de Gini (ano 2000) realizado com sucesso" )

In [ ]:
# ---------- CALCULO DO ÍNDICE DE GINI PARA RENDA DOMICILIAR ---------- #

estados = [ "ac", "al", "am", "ap", "ba", "ce", "df", "es", "go", "ma", "mg", "ms", "mt",
          "pa", "pb", "pe", "pi", "pr", "rj", "rn1", "ro", "rr", "rs", "sc", "se", 
          "sp", "to" ]

results = []

for k in estados:
    
    # Seleciona os estados e municpios
    sql = "SELECT DISTINCT( v0103 ) AS cidade FROM censo_2000.domicilios_%s" % k
    cidades = pd.read_sql_query( sql, conn )

    for i in cidades['cidade']:
        
        # Referenciando estado e cidade (k, i)
        
        # SQL - índice de gini renda domiciliar  
        sql_RD = "SELECT \
                COALESCE( ( v7616 * p001 ), 0 ) AS renda, ( p001 ) AS peso \
                FROM censo_2000.domicilios_%s \
                WHERE v0103 = %d" %( k, i )
        
        # SQL - índice de gini renda per capita
        sql_RP = "SELECT \
                COALESCE( ( v7616 * p001 ), 0 ) AS renda, ( p001 * v7100 ) AS peso \
                FROM censo_2000.domicilios_%s \
                WHERE v0103 = %d" %( k, i )

        # SQL - renda per capita
        sql_Renda = "SELECT \
                    ( SUM( v7616 * p001 ) / SUM( v7100 * p001 ) ) AS RP \
                    FROM censo_2000.domicilios_%s \
                    WHERE v0103 = %d" %( k, i )

        df_Renda = pd.read_sql_query( sql_Renda, conn )
        
        # Ajuste da renda per capita de julho de 2000 para valores em agosto de 2010
        # 99,19% de inflação no INCP
        renda2010 = df_Renda.loc[ 0, 'rp' ] + ( df_Renda.loc[ 0, 'rp' ] * 0.9919037 )
        
        # Obs: A função "gini()" retorna um array com [índice de gini], ...
        # proporção acumulado da população] e [proporção acumulada de renda]
        g_RD = gini( sql_RD )
        g_RP = gini( sql_RP )
        
        # Adiciona valores no array resultado
        # || ID do municipio || Gini RD || Gini RP || Renda2000 || Renda2010 ||
        results.append( [ i,
                         g_RD[ 0 ], 
                         g_RP[ 0 ], 
                         df_Renda.loc[ 0, 'rp' ], 
                         renda2010 ] )

        
# Salva em um arquivo .csv
gini_domicilio = pd.DataFrame( results, columns = 'Cod Gini_RD Gini_RP RendaMedia RendaMedia2010'.split() )
gini_domicilio.to_csv( 'results/gini2000_cidades.csv', sep = '\t' )
        
print( "Processamento do cálculo do índice de Gini (ano 2000) realizado com sucesso" )

In [ ]:
# Ilustração de curva de Lorenz

plotagem(g_RP[ 1 ], g_RP[ 2 ])

## **2. Participação dos benefícios de aposentadorias e pensões**

### **Rendimento de beneficios por estados**
* Renda e percentual para o total dos estados
* Renda e percentual para **aposentadorias** e **pensões** para cada estado
* Renda e percentual para **aposentadorias** e **pensões** até 1 salário mínimo
* Renda e percentual para **aposentadorias** e **pensões** acima de 1 salário mínimo
* Foi utilizado o software [Pentaho Data Integration](https://sourceforge.net/projects/pentaho/) para a realização das consultas na base de dados

In [ ]:
# ---------- RESULTADOS DAS PARTICIPAÇÕES DE BENEFICIOS NO BRASIL - ESTADOS ---------- #

estados = [ "ac", "al", "am", "ap", "ba", "ce", "df", "es", "go", "ma", "mg", "ms", "mt",
          "pa", "pb", "pe", "pi", "pr", "rj", "rn1", "rn2", "ro", "rr", "rs", "sc", "se", 
          "sp", "to" ]

results = []

for k in estados: 
    # Rendimento total do estado
    sql_RT = "SELECT SUM( v4614 * p001 ) AS RT \
            FROM censo_2000.pessoas_%s" %k

    # Rendimento total dos beneficios do estado
    sql_RB = "SELECT SUM( v4573 * p001 ) AS RB \
            FROM censo_2000.pessoas_%s" %k

    # Rendimento total dos beneficios acima de 1 salário minímo
    sql_RB_nSAL = "SELECT SUM( v4573 * p001 ) AS RB_nSAL \
            FROM censo_2000.pessoas_%s WHERE v4573 > 151" %k

    # Rendimento total dos beneficios igual ou menor que 1 salário minímo
    sql_RB_1SAL = "SELECT SUM( v4573 * p001 ) AS RB_1SAL \
            FROM censo_2000.pessoas_%s WHERE v4573 <= 151" %k

    df_RT = pd.read_sql_query( sql_RT, conn )
    df_RB = pd.read_sql_query( sql_RB, conn )
    df_RB_nSAL = pd.read_sql_query( sql_RB_nSAL, conn )
    df_RB_1SAL = pd.read_sql_query( sql_RB_1SAL, conn )

    RT = df_RT.loc[ 0, 'rt' ]
    RB = df_RB.loc[ 0, 'rb' ]
    RB_nSAL = df_RB_nSAL.loc[ 0, 'rb_nsal' ]
    RB_1SAL = df_RB_1SAL.loc[ 0, 'rb_1sal' ]

    results.append( [ k,
                      RT,
                      RB,
                      RB / RT,
                      RB_nSAL,
                      RB_nSAL / RT,
                      RB_1SAL,
                      RB_1SAL / RT] )

#results.append( [ "br", np.sum( results, axis=0 ) ] )

beneficios_2000_uf = pd.DataFrame( results, columns = "UF Renda(Total)\
                                                    Renda(Beneficios) Total(%)\
                                                    Beneficio(>Sal) Beneficio(>Sal)(%)\
                                                    Beneficio(<=Sal) Beneficio(<=Sal)(%)".split() )

beneficios_2000_uf.to_csv( 'results/beneficios2000_uf.csv', sep = '\t' )
        
print( "Processamento do cálculo dos benfícios (ano 2000) realizado com sucesso" )

In [ ]:
# Plotagem da participação de aposentadorias e pensões no total para o Brasil
import matplotlib.pyplot as plt

# Percentual de participação dos beneficios em todo o Brasil
p_total = beneficios_2000_uf['Total(%)'].sum() * 100 

# Percentual de participação dos beneficios <= 1 sal em todo o Brasil
p_um_sal = beneficios_2000_uf['Beneficio(<=Sal)(%)'].sum() * 100 

# Percentual de participação dos beneficios > 1 sal em todo o Brasil
p_mais_um_sal = beneficios_2000_uf['Beneficio(>Sal)(%)'].sum() * 100

p_total_sal = [ ( ( p_um_sal * 100 ) / p_total ), ( ( p_mais_um_sal * 100 ) / p_total ) ] 

labels = 'Até 1 Salário','Acima de 1 Salário'
plt.pie( p_total_sal, colors = [ 'y', 'r' ], autopct = '%1.1f%%' )
plt.title( 'Aposentadorias e pensões - Brasil' )
plt.legend( labels, bbox_to_anchor = ( 1.05, 1 ) )
plt.show()

In [ ]:
# ---------- RESULTADOS DAS PARTICIPAÇÕES DE BENEFICIOS NO BRASIL - CIDADES ---------- #

estados = [ "ac", "al", "am", "ap", "ba", "ce", "df", "es", "go", "ma", "mg", "ms", "mt",
          "pa", "pb", "pe", "pi", "pr", "rj", "rn1", "ro", "rr", "rs", "sc", "se", 
          "sp", "to" ]

results = []

for k in estados: 

    # Seleciona os estados e municpios
    sql = "SELECT DISTINCT( v1103 ) AS cidade FROM censo_2000.pessoas_%s WHERE v1103 <> 0" % k
    cidades = pd.read_sql_query( sql, conn )
    
    for i in cidades['cidade']:
        
        # Referenciando estado e cidade (k, i)
        
        # Rendimento total do estado
        sql_RT = "SELECT COALESCE( SUM( v4614 * p001 ), 0 ) AS RT \
                FROM censo_2000.pessoas_%s WHERE v1103 = %s" %( k, i )

        # Rendimento total dos beneficios do estado
        sql_RB = "SELECT COALESCE( SUM( v4573 * p001 ), 0 ) AS RB \
                FROM censo_2000.pessoas_%s WHERE v4573 <> 0 AND v1103 = %s" %( k, i )

        # Rendimento total dos beneficios acima de 1 salário minímo
        sql_RB_nSAL = "SELECT COALESCE( SUM( v4573 * p001 ), 0 ) AS RB_nSAL \
                FROM censo_2000.pessoas_%s WHERE v4573 > 151 AND v4573 <> 0 AND v1103 = %s" %( k, i )

        # Rendimento total dos beneficios igual ou menor que 1 salário minímo
        sql_RB_1SAL = "SELECT COALESCE( SUM( v4573 * p001 ), 0 ) AS RB_1SAL \
                FROM censo_2000.pessoas_%s WHERE v4573 <= 151 AND v4573 <> 0 AND v1103 = %s" %( k, i )

        df_RT = pd.read_sql_query( sql_RT, conn )
        df_RB = pd.read_sql_query( sql_RB, conn )
        df_RB_nSAL = pd.read_sql_query( sql_RB_nSAL, conn )
        df_RB_1SAL = pd.read_sql_query( sql_RB_1SAL, conn )

        RT = df_RT.loc[ 0, 'rt' ]
        RB = df_RB.loc[ 0, 'rb' ]
        RB_nSAL = df_RB_nSAL.loc[ 0, 'rb_nsal' ]
        RB_1SAL = df_RB_1SAL.loc[ 0, 'rb_1sal' ]
        
        results.append( [ k,
                          i,
                          RT,
                          RB,
                          RB / RT,
                          RB_nSAL,
                          RB_nSAL / RT,
                          RB_1SAL,
                          RB_1SAL / RT] )



beneficios_2000_cid = pd.DataFrame( results, columns = "COD UF Renda(Total)\
                                                    Renda(Beneficios) Total(%)\
                                                    Beneficio(>Sal) Beneficio(>Sal)(%)\
                                                    Beneficio(<=Sal) Beneficio(<=Sal)(%)".split() )

beneficios_2000_cid.to_csv( 'results/beneficios2000_cidades.csv', sep = '\t' )

print( "Processamento do cálculo dos benfícios para cidades (ano 2000) realizado com sucesso" )

## **3. Medida de progressividade dos benefícios**

### **Progressividade dos benefícios por estados** 

In [ ]:
estados = [ "ac", "al", "am", "ap", "ba", "ce", "df", "es", "go", "ma", "mg", "ms", "mt",
          "pa", "pb", "pe", "pi", "pr", "rj", "rn1", "ro", "rr", "rs", "sc", "se", 
          "sp", "to" ]

prog_2000_uf = pd.DataFrame( data = [], columns = 'uf gini(g) parcela(ch) p_total p_1sal p_nsal'.split() )

for k in estados:

    sql = "SELECT \
            COALESCE( v4614, 0 ) AS renda_tot, \
            COALESCE( ROUND( p001 ), 0 ) AS peso_tot, \
            COALESCE( v4573, 0 ) AS ben_tot, \
            COALESCE( CASE WHEN v4573 <= 151 THEN v4573 ELSE 0 END, 0) AS ben_1sal, \
            COALESCE( CASE WHEN v4573 > 151 THEN v4573 ELSE 0 END, 0) AS ben_nsal \
           FROM censo_2000.pessoas_%s WHERE v4614 <> 0 AND v4614 IS NOT NULL ORDER BY v4614 ASC" %k
    
    df = pd.read_sql_query( sql, conn )     
    df = df.sort_values( 'renda_tot', kind = 'mergesorted')

    df = df.reindex( df.index.repeat( pd.to_numeric( df.peso_tot, downcast='integer') ) )
    
    #df_tot[ 'pop_cum' ] = np.arange( len( df_tot ) )
    #df_tot[ 'pop_cum' ] = df_tot.pop_cum.cumsum() / df_tot.pop_cum.sum()
    df[ 'pop_cum' ] = np.ones( len( df ) )
    df[ 'pop_cum' ] = df.pop_cum.cumsum() / df.pop_cum.size
    df[ 'renda_cum' ] = df.renda_tot.cumsum() / df.renda_tot.sum()
    df[ 'ben_cum' ] = df.ben_tot.cumsum() / df.ben_tot.sum()
    
    g = 2 * ( 0.5 - np.trapz( df.renda_cum, df.pop_cum ) )
    ch1 = 2 * ( 0.5 - np.trapz( df.ben_cum, df.pop_cum ) )
    p1 = g - ch1
    
    df[ 'ben_cum' ] = df.ben_1sal.cumsum() / df.ben_1sal.sum()
    ch2 = 2 * ( 0.5 - np.trapz( df.ben_cum, df.pop_cum ) )
    p2 = g - ch2
    
    df[ 'ben_cum' ] = df.ben_nsal.cumsum() / df.ben_nsal.sum()
    ch3 = 2 * ( 0.5 - np.trapz( df.ben_cum, df.pop_cum ) )
    p3 = g - ch3
    
    prog_2000_uf = prog_2000_uf.append( {'uf': k,
                                        'gini(g)': g, 
                                        'parcela(ch)': ch1, 
                                        'p_total': p1, 
                                        'p_1sal': p2, 
                                        'p_nsal': p3},
                                      ignore_index = True)
    
prog_2000_uf

In [ ]:
prog_2000_uf.to_csv( 'results/progressividade_2000_uf.csv', sep = '\t' )

print( 'Processamento da progressividade para estados concluída' )

### **Progressividade dos benefícios por cidades**

In [ ]:
estados = [ "ac", "al", "am", "ap", "ba", "ce", "df", "es", "go", "ma", "mg", "ms", "mt",
          "pa", "pb", "pe", "pi", "pr", "rj", "rn1", "ro", "rr", "rs", "sc", "se", 
          "sp", "to" ]

prog_2000_cidade = pd.DataFrame( data = [], columns = 'uf cod gini(g) parcela(ch) p_total p_1sal p_nsal'.split() )

for k in estados:

    # Seleciona os estados e municpios
    sql = "SELECT DISTINCT( v1103 ) AS cidade FROM censo_2000.pessoas_%s WHERE v1103 <> 0" % k
    cidades = pd.read_sql_query( sql, conn )
    
    for i in cidades['cidade']:
        sql = "SELECT \
                COALESCE( v4614, 0 ) AS renda_tot, \
                COALESCE( ROUND( p001 ), 0 ) AS peso_tot, \
                COALESCE( v4573, 0 ) AS ben_tot, \
                COALESCE( CASE WHEN v4573 <= 151 THEN v4573 ELSE 0 END, 0) AS ben_1sal, \
                COALESCE( CASE WHEN v4573 > 151 THEN v4573 ELSE 0 END, 0) AS ben_nsal \
               FROM censo_2000.pessoas_%s WHERE v4614 <> 0 AND v4614 IS NOT NULL AND v1103 = %s ORDER BY v4614 ASC" %(k,i)

        df = pd.read_sql_query( sql, conn )     
        df = df.sort_values( 'renda_tot', kind = 'mergesorted')

        df = df.reindex( df.index.repeat( pd.to_numeric( df.peso_tot, downcast='integer') ) )

        #df_tot[ 'pop_cum' ] = np.arange( len( df_tot ) )
        #df_tot[ 'pop_cum' ] = df_tot.pop_cum.cumsum() / df_tot.pop_cum.sum()
        df[ 'pop_cum' ] = np.ones( len( df ) )
        df[ 'pop_cum' ] = df.pop_cum.cumsum() / df.pop_cum.size
        df[ 'renda_cum' ] = df.renda_tot.cumsum() / df.renda_tot.sum()
        df[ 'ben_cum' ] = df.ben_tot.cumsum() / df.ben_tot.sum()

        g = 2 * ( 0.5 - np.trapz( df.renda_cum, df.pop_cum ) )
        ch1 = 2 * ( 0.5 - np.trapz( df.ben_cum, df.pop_cum ) )
        p1 = g - ch1

        df[ 'ben_cum' ] = df.ben_1sal.cumsum() / df.ben_1sal.sum()
        ch2 = 2 * ( 0.5 - np.trapz( df.ben_cum, df.pop_cum ) )
        p2 = g - ch2

        df[ 'ben_cum' ] = df.ben_nsal.cumsum() / df.ben_nsal.sum()
        ch3 = 2 * ( 0.5 - np.trapz( df.ben_cum, df.pop_cum ) )
        p3 = g - ch3

        prog_2000_cidade = prog_2000_cidade.append( {'uf': k,
                                                    'cod': i,
                                                    'gini(g)': g, 
                                                    'parcela(ch)': ch1, 
                                                    'p_total': p1, 
                                                    'p_1sal': p2, 
                                                    'p_nsal': p3},
                                                      ignore_index = True)

prog_2000_cidade = prog_2000_cidade.fillna( 0 )
prog_2000_cidade

In [ ]:
prog_2000_cidade.to_csv( 'results/progressividade_2000_cidades.csv', sep = '\t' )

print( 'Processamento da progressividade para cidades concluída' )